# Token Classification

In [1]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric

c:\Users\baiet\Desktop\NLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading Dataset

In [2]:
def filter_null_rows(example):
    '''Checking and removing examples with None values in 'text' or 'label'.'''
    return example['text'] is not None and example['label'] is not None

# Preparing the dataset
dataset_path = "eriktks/conll2003"
dataset = load_dataset(dataset_path).remove_columns(['id', 'pos_tags', 'chunk_tags'])
num_labels = len(dataset["train"].features["ner_tags"].feature.names)

# Split into train, validation and test
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

c:\Users\baiet\Desktop\NLP\.venv\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for eriktks/conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eriktks/conll2003
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


### Fine-Tuning

In [3]:
def Training_Tok_Clas(model_name, dataset_path, train, val):

    def set_seed(seed):
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
 
    set_seed(123)# Set the seed to ensure reproducibility

    # Load the pre-trained tokenizer and model for Token classification
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)
    
    def tokenize_and_align_labels(examples):
        '''Function to tokenize and align labels for token classification'''
        tokenized_inputs = tokenizer(examples["tokens"], truncation=True,padding='max_length', is_split_into_words=True, max_length= 100)
        labels = []
        for i, label in enumerate(examples[f"ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    tok_train = train.map(tokenize_and_align_labels, batched=True)
    tok_val = val.map(tokenize_and_align_labels, batched=True)

    # Define training arguments for the Trainer
    training_args = TrainingArguments(
        seed=123,
        data_seed=123,
        output_dir=f"./results_{model_name.split('/')[1]}_{dataset_path.split('/')[1]}", # Output directory for results
        evaluation_strategy='epoch',  # Evaluate the model at the end of each epoch
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01)  # Weight decay for regularization
    
    # Initialize the Trainer with the model, training arguments, and datasets
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tok_train,
        eval_dataset=tok_val)
    
    trainer.train()
    
    # Save the trained model and tokenizer to the specified directory
    model.save_pretrained(f"./{model_name.split('/')[1]}_{dataset_path.split('/')[1]}")
    tokenizer.save_pretrained(f"./{model_name.split('/')[1]}_{dataset_path.split('/')[1]}")

In [6]:
Training_Tok_Clas("squeezebert/squeezebert-uncased", "eriktks/conll2003", train_dataset, val_dataset)

c:\Users\baiet\Desktop\NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of SqueezeBertForTokenClassification were not initialized from the model checkpoint at squeezebert/squeezebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\baiet\Desktop\NLP\.venv\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 19%|█▉        | 500/2634 [01:04<04:31,  7.87it/s]

{'loss': 0.7718, 'grad_norm': 1.0940091609954834, 'learning_rate': 1.6203492786636296e-05, 'epoch': 0.57}


 33%|███▎      | 879/2634 [02:00<49:40,  1.70s/it]

{'eval_loss': 0.26540929079055786, 'eval_runtime': 6.824, 'eval_samples_per_second': 476.262, 'eval_steps_per_second': 29.895, 'epoch': 1.0}


 38%|███▊      | 1000/2634 [02:16<03:25,  7.95it/s]

...

{'loss': 0.1803, 'grad_norm': 0.8626317977905273, 'learning_rate': 4.8139711465451785e-06, 'epoch': 2.28}


 95%|█████████▍| 2500/2634 [05:35<00:16,  7.95it/s]

{'loss': 0.1577, 'grad_norm': 1.0971603393554688, 'learning_rate': 1.0174639331814731e-06, 'epoch': 2.85}


                                                   
100%|██████████| 2634/2634 [05:59<00:00,  7.33it/s]


{'eval_loss': 0.14918379485607147, 'eval_runtime': 6.6378, 'eval_samples_per_second': 489.618, 'eval_steps_per_second': 30.733, 'epoch': 3.0}
{'train_runtime': 359.4554, 'train_samples_per_second': 117.186, 'train_steps_per_second': 7.328, 'train_loss': 0.32506431660691987, 'epoch': 3.0}


In [5]:
Training_Tok_Clas("google-bert/bert-base-uncased", "eriktks/conll2003", train_dataset, val_dataset)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 3250/3250 [00:00<00:00, 3740.93 examples/s]
c:\Users\baiet\Desktop\NLP\.venv\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/2634 [00:00<?, ?it/s]c:\Users\baiet\Desktop\NLP\.venv\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
 19%|█▉        | 500/2634 [01:13<05:13,  6.8

{'loss': 0.2305, 'grad_norm': 2.7367660999298096, 'learning_rate': 1.6203492786636296e-05, 'epoch': 0.57}


 33%|███▎      | 879/2634 [02:21<1:22:49,  2.83s/it]

{'eval_loss': 0.06300098448991776, 'eval_runtime': 8.9823, 'eval_samples_per_second': 361.821, 'eval_steps_per_second': 22.711, 'epoch': 1.0}


 38%|███▊      | 1000/2634 [02:39<03:57,  6.88it/s] 

...

{'loss': 0.033, 'grad_norm': 1.7722903490066528, 'learning_rate': 4.8139711465451785e-06, 'epoch': 2.28}


 95%|█████████▍| 2500/2634 [06:48<00:19,  6.82it/s]

{'loss': 0.0258, 'grad_norm': 0.18871574103832245, 'learning_rate': 1.0174639331814731e-06, 'epoch': 2.85}


100%|██████████| 2634/2634 [07:25<00:00,  5.91it/s]


{'eval_loss': 0.05526485666632652, 'eval_runtime': 8.978, 'eval_samples_per_second': 361.994, 'eval_steps_per_second': 22.722, 'epoch': 3.0}
{'train_runtime': 445.4917, 'train_samples_per_second': 94.554, 'train_steps_per_second': 5.913, 'train_loss': 0.07933414371609235, 'epoch': 3.0}


### Testing

In [10]:
def Testing_Tok_Clas(model_name, dataset_path, test):
    
    model = AutoModelForTokenClassification.from_pretrained(f"./{model_name.split('/')[1]}_{dataset_path.split('/')[1]}")
    tokenizer = AutoTokenizer.from_pretrained(f"./{model_name.split('/')[1]}_{dataset_path.split('/')[1]}")

    def set_seed(seed):
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    set_seed(123)# Set the seed to ensure reproducibility
    
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(examples["tokens"], truncation=True,padding='max_length', is_split_into_words=True, max_length= 100)
        labels = []
        for i, label in enumerate(examples[f"ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    tok_test = test.map(tokenize_and_align_labels, batched=True)
    label_list = dataset["train"].features["ner_tags"].feature.names

    metric = load_metric("seqeval")

    def compute_metrics(eval_preds):
        '''Cmputing evaluation metrics'''
    
        pred_logits, labels = eval_preds 
        
        pred_logits = np.argmax(pred_logits, axis=2) 

        predictions = [ 
            [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
            for prediction, label in zip(pred_logits, labels)] 
        
        true_labels = [ 
        [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels)] 
        results = metric.compute(predictions=predictions, references=true_labels) 
        return { 
            "precision": results["overall_precision"], 
            "recall": results["overall_recall"], 
            "f1": results["overall_f1"], 
            "accuracy": results["overall_accuracy"]}
    
    # Define training arguments for the Trainer
    testing_args = TrainingArguments(
        seed=123,
        data_seed=123,
        output_dir="./results", # Output directory for results
        use_cpu=True) # Use CPU for testing (set to False if GPU is available)

    trainer = Trainer(
        args=testing_args,
        model=model,
        eval_dataset=tok_test,
        compute_metrics=compute_metrics)

    return trainer.predict(tok_test)[-1]

In [11]:
Testing_Tok_Clas("squeezebert/squeezebert-uncased", "eriktks/conll2003", test_dataset)

c:\Users\baiet\Desktop\NLP\.venv\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 432/432 [02:52<00:00,  2.50it/s]


{'test_loss': 0.17675328254699707,
 'test_precision': 0.8467713787085515,
 'test_recall': 0.8655012486621477,
 'test_f1': 0.8560338743824982,
 'test_accuracy': 0.9674151670728967,
 'test_runtime': 172.9217,
 'test_samples_per_second': 19.969,
 'test_steps_per_second': 2.498}

In [12]:
Testing_Tok_Clas("google-bert/bert-base-uncased", "eriktks/conll2003", test_dataset)

Map: 100%|██████████| 3453/3453 [00:00<00:00, 4689.11 examples/s]
c:\Users\baiet\Desktop\NLP\.venv\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 432/432 [04:59<00:00,  1.44it/s]


{'test_loss': 0.12570184469223022,
 'test_precision': 0.8886158886158886,
 'test_recall': 0.9031989535022,
 'test_f1': 0.8958480773767398,
 'test_accuracy': 0.9756261992428564,
 'test_runtime': 300.034,
 'test_samples_per_second': 11.509,
 'test_steps_per_second': 1.44}